In [22]:
import pandas as pd

df_tags = pd.read_csv('../data/raw/ml-25m/tags.csv')
df_gscores = pd.read_csv('../data/raw/ml-25m/genome-scores.csv')
df_gtags = pd.read_csv('../data/raw/ml-25m/genome-tags.csv')
df_links = pd.read_csv('../data/raw/ml-25m/links.csv')
df_movies = pd.read_csv('../data/raw/ml-25m/movies.csv')
df_ratings = pd.read_csv('../data/raw/ml-25m/ratings.csv')

In [51]:
df_gscores.head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [48]:
#The same movieId refers to the same movie between `ratings.csv`, `tags.csv`, `movies.csv`, and `links.csv`
df = pd.merge(df_movies, df_ratings, how="left", on=['movieId'])
df = pd.merge(df, df_tags, on=['movieId', 'userId', 'timestamp'], how='left')
df = pd.merge(df, df_gtags, on=['tag'], how='left')
df = pd.merge(df, df_gscores, on=['tagId', 'movieId'], how='left')

In [53]:
df['genres'].value_counts()

genres
Drama                                   1766138
Comedy                                  1513066
Comedy|Romance                           902164
Drama|Romance                            796458
Comedy|Drama                             721506
                                         ...   
Adventure|Children|Drama|War                  1
Adventure|Fantasy|Western                     1
Action|Comedy|Documentary|Thriller            1
Adventure|Fantasy|Sci-Fi|Western              1
Comedy|Horror|Mystery|Sci-Fi|Western          1
Name: count, Length: 1639, dtype: int64

##### Deadlines:
- Report 1. Data viz (5 graphs) + preprocessing. Friday, May 17
- Report 2. Modelling. Friday, June 14
- Final report. Friday, June 21
